### Methods for non-linear minimization:
- Coordinate descent
- Gradient descent
- Newton method
- BFGS

### Methods for non-linear least-squares:
- Gauss-newton
- Levenberg-Marquardt

In [ ]:
import autograd.numpy as np
from autograd import grad as auto_grad, jacobian as auto_jacobian
import scipy

import matplotlib.pyplot as plt
import seaborn

In [ ]:
def solve_square_with_qr(A, b):
    """Solve system of equations with square (real values) matrix A: A * x = b"""""
    Q, R = np.linalg.qr(A)
    x = scipy.linalg.solve_triangular(R, Q.T @ b)
    return x


def solve_symm_with_chol(A, b):
    """Solve system of equations with symmetric (hermitian) matrix A: A * x = b"""""
    c, lower = scipy.linalg.cho_factor(A)
    x = scipy.linalg.cho_solve((c, lower), b)
    return x

In [ ]:
np.random.seed(41)

n0 = 1
def objective_fn0(x):
    y = -x[..., 0]**2
    return y

grad_fn0 = auto_grad(objective_fn0)
hessian_fn0 = auto_jacobian(grad_fn0)

n1 = 1
def objective_fn1(x):
    y = -100*x[..., 0] + 0.1*x[..., 0]**3 + 0.001*x[..., 0]**4
    return y

grad_fn1 = auto_grad(objective_fn1)
hessian_fn1 = auto_jacobian(grad_fn1)

n2 = 2
def objective_fn2(x):
    y = x[..., 0]**2 - 20*x[..., 0] + 0.4*x[..., 1] + 50*x[..., 1]
    return y

grad_fn2 = auto_grad(objective_fn2)
hessian_fn2 = auto_jacobian(grad_fn2)

n3 = 2
def objective_fn3(x):
    y = x[..., 0]**4 + x[..., 1]**2
    return y

grad_fn3 = auto_grad(objective_fn3)
hessian_fn3 = auto_jacobian(grad_fn3)

n4 = 100
x_in4 = np.linspace(0, 5, n4)
y_out4 = x_in4**2 + 1 * np.random.randn(n4)
def residuals_fn4(x):
    y = x[..., 0] + x_in4 * x[..., 1] + x_in4**2 * x[..., 2]  - y_out4
    return y
def objective_fn4(x):
    y = np.sum(residuals_fn4(x)**2, axis=x.ndim - 1)
    return y

residuals_jacobian_fn4 = auto_jacobian(residuals_fn4)
grad_fn4 = auto_grad(objective_fn4)
hessian_fn4 = auto_jacobian(grad_fn4)

n5 = 20
x_in5 = np.linspace(0, 100, n5)
y_out5 = 100 * np.cos(102 * x_in5) + 102 * np.sin(100 * x_in5)
def residuals_fn5(x):
    y = x[..., 0] * np.cos(x[..., 1] * x_in5) + x[..., 1] * np.sin(x[..., 0] * x_in5) - y_out5
    return y
def objective_fn5(x):
    y = np.sum(residuals_fn5(x)**2, axis=x.ndim - 1)
    return y

residuals_jacobian_fn5 = auto_jacobian(residuals_fn5)
grad_fn5 = auto_grad(objective_fn5)
hessian_fn5 = auto_jacobian(grad_fn5)

In [ ]:
def min_coordinate_descent(objective_fn, x0, alpha,
                           max_dobj=0., min_step_size=1e-15, obj_tol=1e-12, max_iterations=10**6):
    x = np.array(x0)
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    coord_index = 0
    coord_dir = +1
    no_progress_count = 0
    while no_progress_count < 2 * len(x) and iteration < max_iterations:
        prev_obj_value = obj_value
        step_dir = np.zeros(x.shape)
        step_dir[coord_index] = coord_dir
        step_size = alpha
        dobj = float("inf")
        x_next = x
        while dobj > max_dobj and step_size >= min_step_size:
            x_next = x + step_size * step_dir
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            step_size *= 0.5
        if np.abs(dobj) <= obj_tol:
            no_progress_count += 1
        else:
            no_progress_count = 0
        x = x_next
        iteration += 1
        coord_index += 1
        if coord_index >= x.shape[-1]:
            coord_index = 0
            coord_dir = -coord_dir
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "step_dir": step_dir,
        "step_size": step_size,
        "no_progress_count": no_progress_count,
    }
    return x, info


In [ ]:
def min_gradient_descent(objective_fn, grad_fn, x0, alpha, max_grad_norm,
                         max_dobj=0., min_step_size=1e-15, obj_tol=1e-12, max_iterations=10**6):
    x = np.array(x0)
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        prev_obj_value = obj_value
        grad = grad_fn(x)
        grad_norm = np.linalg.norm(grad)
        if grad_norm > max_grad_norm:
            grad *= max_grad_norm / grad_norm
        step_dir = -grad
        step_size = alpha
        dobj = float("inf")
        while dobj > max_dobj and step_size >= min_step_size:
            x_next = x + step_size * step_dir
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            step_size *= 0.5
        x = x_next
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "grad_norm": grad_norm,
        "step_dir": step_dir,
        "step_size": step_size,
    }
    return x, info


In [ ]:
def min_newton_method(objective_fn, grad_fn, hessian_fn, x0, alpha0, max_step_norm,
                      max_dobj=0., min_step_size=1e-15, obj_tol=1e-15, max_iterations=10**6):
    x = np.array(x0)
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        prev_obj_value = obj_value
        grad = grad_fn(x)
        hessian = hessian_fn(x)
        # Hessian modification
        eigvals = np.linalg.eigvalsh(hessian)
        if eigvals[0] <= 0:
            hessian += (1 + np.abs(eigvals[0])) * np.eye(hessian.shape[0])
        step_dir = solve_symm_with_chol(hessian, -grad)
#         step_dir = -np.linalg.inv(hessian) @ grad
        alpha = alpha0
        step = alpha * step_dir
        step_norm = np.linalg.norm(step)
        if step_norm > max_step_norm:
            step_dir *= max_step_norm / (step_norm)
        x_next = x
        dobj = float("inf")
        while dobj > max_dobj and step_norm >= min_step_size:
            step = alpha * step_dir
            step_norm = np.linalg.norm(step)
            x_next = x + step
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            alpha *= 0.5
        x = x_next
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "hessian": hessian,
        "alpha": alpha,
        "step_dir": step_dir,
        "step": step,
    }
    return x, info

In [ ]:
def min_bfgs(objective_fn, grad_fn, x0, alpha0, max_step_norm,
                      max_dobj=0., min_step_size=1e-15, divide_tol=1e-15, obj_tol=1e-15, max_iterations=10**6):
    x = np.array(x0)
    #B = np.eye(x.shape[-1])
    B_inv = np.eye(x.shape[-1])
    iteration = 0
    dobj = float("inf")
    obj_value = objective_fn(x)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        prev_obj_value = obj_value
        grad = grad_fn(x)
#         step_dir = solve_symm_with_qr(B, -grad)
        #step_dir = -np.linalg.inv(B) @ grad
        # Hessian modification
        eigvals = np.linalg.eigvalsh(B_inv)
        if eigvals[0] <= 0:
            B_inv += (1 + np.abs(eigvals[0])) * np.eye(B_inv.shape[0])
        step_dir = -B_inv @ grad
        alpha = alpha0
        step = alpha * step_dir
        step_norm = np.linalg.norm(step)
        if step_norm > max_step_norm:
            step_dir *= max_step_norm / (step_norm)
        x_next = x
        dobj = float("inf")
        while dobj > max_dobj and step_norm >= min_step_size:
            step = alpha * step_dir
            step_norm = np.linalg.norm(step)
            x_next = x + step
            obj_value = objective_fn(x_next)
            dobj = obj_value - prev_obj_value
            alpha *= 0.5
        x = x_next
        step = step[..., np.newaxis]
        y = (grad_fn(x) - grad)[..., np.newaxis]
        B_inv_step1_nom = (step.T @ y + y.T @ B_inv @ y) * (step @ step.T)
        B_inv_step1_denom = (step.T @ y) ** 2
        B_inv_step1 = B_inv_step1_nom / B_inv_step1_denom
        B_inv_step2 = - (B_inv @ y @ step.T + step @ y.T @ B_inv) / (step.T @ y)
        B_inv += B_inv_step1 + B_inv_step2
#         B_step1_denom = y.T @ step
#         if np.abs(B_step1_denom) >= divide_tol:
#             B_step1 = (y @ y.T) / B_step1_denom
#             B_step2 = - (B @ step @ step.T @ B) / (step.T @ B @ step)
#             B = B + B_step1 + B_step2
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "alpha": alpha,
        "step_dir": step_dir,
        "step": step,
#         "B": B,
        "B_inv": B_inv,
    }
    return x, info


In [ ]:
def min_gauss_newton(residuals_fn, residuals_jacobian_fn, x0, obj_tol=1e-12, max_iterations=10**6):
    x = np.array(x0)
    iteration = 0
    dobj = float("inf")
    residuals = residuals_fn(x)
    obj_value = np.sum(residuals**2)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        grad = residuals_jacobian_fn(x)
        pseudo_inv = np.linalg.pinv(grad.T @ grad)
        step_dir = (pseudo_inv @ grad.T) @ residuals
        prev_obj_value = obj_value
        step_size = 1
        x = x - step_size * step_dir
        residuals = residuals_fn(x)
        obj_value = np.sum(residuals**2)
        dobj = obj_value - prev_obj_value
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "step_dir": step_dir,
        "step_size": step_size,
    }
    return x, info


In [ ]:
def min_levenberg_marquardt(residuals_fn, residuals_jacobian_fn, x0, lambda0=1., v=2.,
                            obj_tol=1e-12, max_iterations=10**6):
    x = np.array(x0)
    lambda_ = lambda0
    iteration = 0
    dobj = float("inf")
    residuals = residuals_fn(x)
    obj_value = np.sum(residuals**2)
    while np.abs(dobj) > obj_tol and iteration < max_iterations:
        prev_obj_value = obj_value
        grad = residuals_jacobian_fn(x)
        grad_grad = grad.T @ grad
        while True:  # Should max iteration criterion here
            lambda1 = lambda_
            lambda2 = lambda_ / v
            H1 = grad_grad + lambda1 * np.diag(np.diag(grad_grad))
            H2 = grad_grad + lambda2 * np.diag(np.diag(grad_grad))
            pseudo_inv1 = np.linalg.pinv(H1)
            pseudo_inv2 = np.linalg.pinv(H2)
            step_dir1 = (pseudo_inv1 @ grad.T) @ residuals
            step_dir2 = (pseudo_inv2 @ grad.T) @ residuals
            x_next1 = x - step_dir1
            x_next2 = x - step_dir2
            residuals1 = residuals_fn(x_next1)
            residuals2 = residuals_fn(x_next2)
            obj_value1 = np.sum(residuals1**2)
            obj_value2 = np.sum(residuals2**2)
            dobj1 = obj_value1 - prev_obj_value
            dobj2 = obj_value2 - prev_obj_value
            if dobj1 > 0 and dobj2 > 0:
                lambda_ *= v
            elif dobj2 < 0:
                lambda_ = lambda2
                x = x_next2
                step_dir = step_dir2
                residuals = residuals2
                obj_value = obj_value2
                dobj = dobj2
                break
            else:
                x = x_next1
                step_dir = step_dir1
                residuals = residuals1
                obj_value = obj_value1
                dobj = dobj1
                break
        iteration += 1
    info = {
        "obj_value": obj_value,
        "dobj": dobj,
        "obj_tol": obj_tol,
        "iteration": iteration,
        "max_iterations": max_iterations,
        "grad": grad,
        "step_dir": step_dir,
    }
    return x, info


In [ ]:
# Minimize function #0 with gradient descent
x0 = np.array([2.])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 100
x_min, info = min_gradient_descent(objective_fn0, grad_fn0, x0, alpha,
                                   max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min), +np.abs(x_min), 100)[:, np.newaxis]
plt.plot(x, objective_fn0(x))
plt.plot([x_min], [objective_fn0(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #0 with newton method
x0 = np.array([2.])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 100
x_min, info = min_newton_method(objective_fn0, grad_fn0, hessian_fn0, x0, alpha,
                                   max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min), +np.abs(x_min), 100)[:, np.newaxis]
plt.plot(x, objective_fn0(x))
plt.plot([x_min], [objective_fn0(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #0 with BFGS
x0 = np.array([2.])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 100
x_min, info = min_bfgs(objective_fn0, grad_fn0, x0, alpha,
                                   max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min), +np.abs(x_min), 100)[:, np.newaxis]
plt.plot(x, objective_fn0(x))
plt.plot([x_min], [objective_fn0(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with scipy BFGS
x0 = np.array([1.])
import scipy.optimize
res = scipy.optimize.minimize(objective_fn0, x0, method="BFGS", jac=grad_fn0)
x_min = res.x
print("minimum x: {}".format(x_min))
print("objective value: {}".format(res.fun))

# Plot function and found minimum
plt.figure()
x = np.linspace(-np.abs(x_min), +np.abs(x_min), 100)[:, np.newaxis]
plt.plot(x, objective_fn0(x))
plt.plot([x_min], [objective_fn0(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with coordinate descent
x0 = np.array([+1000.])
# x0 = np.array([0.])
alpha = 1e6
max_iterations = 1000
x_min, info = min_coordinate_descent(objective_fn1, x0, alpha, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print(info)

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with gradient descent
x0 = np.array([-20.])
# x0 = np.array([0.])
alpha = 1e6
max_step_norm = float("inf")
x_min, info = min_gradient_descent(objective_fn1, grad_fn1, x0, alpha, max_step_norm)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with newton method
x0 = np.array([+1000.])
# x0 = np.array([0.])
alpha = 1e6
max_grad_norm = float("inf")
max_iterations = 200
x_min, info = min_newton_method(objective_fn1, grad_fn1, hessian_fn1, x0, alpha,
                                max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print(info)

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with BFGS
x0 = np.array([1000.])
alpha = 1e6
max_step_norm = float("inf")
max_iterations = 1000
x_min, info = min_bfgs(objective_fn1, grad_fn1, x0, alpha,
                       max_step_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))
print(info)

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with scipy BFGS
x0 = np.array([-25.])
# x0 = np.array([0.])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 500
import scipy.optimize
res = scipy.optimize.minimize(objective_fn1, x0, method="BFGS", jac=grad_fn1)
x_min = res.x
print("minimum x: {}".format(x_min))
print("objective value: {}".format(res.fun))

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Minimize function #1 with scipy Newton-CG
x0 = np.array([-20.])
# x0 = np.array([0.])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 500
import scipy.optimize
res = scipy.optimize.minimize(objective_fn1, x0, method="Newton-CG", jac=grad_fn1)
x_min = res.x
print("minimum x: {}".format(x_min))
print("objective value: {}".format(res.fun))

# Plot function and found minimum
plt.figure()
x = np.linspace(-90, +30, 100)[:, np.newaxis]
plt.plot(x, objective_fn1(x))
plt.plot([x_min], [objective_fn1(x_min)], 'rx')
plt.show()

In [ ]:
# Function fitting #4 with gradient descent

x0 = np.array([0.0, 0.0, 0.0])
alpha = 0.1
max_grad_norm = 1.
max_iterations = 1000
x_min, info = min_gradient_descent(objective_fn4, grad_fn4, x0, alpha, max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot data and fitted function
plt.figure()
plt.plot(x_in4, x_min[0] + x_min[1]*x_in4 + x_min[2]*x_in4**2, 'b-')
plt.plot(x_in4, y_out4, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

In [ ]:
# Function fitting #4 with newton method

x0 = np.array([0.0, 0.0, 0.0])
alpha = 10000.
max_grad_norm = float("inf")
max_iterations = 1000
x_min, info = min_newton_method(objective_fn4, grad_fn4, hessian_fn4, x0, alpha,
                                max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot data and fitted function
plt.figure()
plt.plot(x_in4, x_min[0] + x_min[1]*x_in4 + x_min[2]*x_in4**2, 'b-')
plt.plot(x_in4, y_out4, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

In [ ]:
# Function fitting #4 with BFGS

x0 = np.array([0.0, 0.0, 0.0])
alpha = 10000
max_step_norm = float("inf")
max_iterations = 10000
x_min, info = min_bfgs(objective_fn4, grad_fn4, x0, alpha,
                       max_step_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot data and fitted function
plt.figure()
plt.plot(x_in4, x_min[0] + x_min[1]*x_in4 + x_min[2]*x_in4**2, 'b-')
plt.plot(x_in4, y_out4, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

In [ ]:
# Function fitting #4 with gauss newton

x0 = np.array([0.0, 0.0, 0.0])
max_iterations = 100
# print(residuals_fn4(x0).shape)
# print(residuals_jacobian_fn4(x0).shape)
x_min, info = min_gauss_newton(residuals_fn4, residuals_jacobian_fn4, x0, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot data and fitted function
plt.figure()
plt.plot(x_in4, x_min[0] + x_min[1]*x_in4 + x_min[2]*x_in4**2, 'b-')
plt.plot(x_in4, y_out4, 'r-')
plt.legend(["Fit", "Data"])
plt.show()
objective_fn4(x_min)

In [ ]:
# Function fitting #4 with levenberg-marquardt

x0 = np.array([0.0, 0.0, 0.0])
max_iterations = 50
# print(residuals_fn4(x0).shape)
# print(residuals_jacobian_fn4(x0).shape)
x_min, info = min_levenberg_marquardt(residuals_fn4, residuals_jacobian_fn4, x0, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(info["obj_value"]))

# Plot data and fitted function
plt.figure()
plt.plot(x_in4, x_min[0] + x_min[1]*x_in4 + x_min[2]*x_in4**2, 'b-')
plt.plot(x_in4, y_out4, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

In [ ]:
# Plot function #5 along both dimensions

xbest = np.array([100., 102.0])

plt.figure()
X = np.linspace(xbest[0] - 1, xbest[0] + 1, 500)
Y = np.array([objective_fn5(np.array([x, xbest[1]])) for x in X])
plt.plot(X, Y)
plt.show()

plt.figure()
X = np.linspace(xbest[1] - 1, xbest[1] + 1, 500)
Y = np.array([objective_fn5(np.array([xbest[0], x])) for x in X])
plt.plot(X, Y)
plt.show()


In [ ]:
# Function fitting #5 with gradient descent

x0 = np.array([100.01, 102.01])
alpha = 0.1
max_grad_norm = 10.
max_iterations = 1000
x_min, info = min_gradient_descent(objective_fn5, grad_fn5, x0,
                                   alpha, max_grad_norm, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(objective_fn5(x_min)))

# Plot data and fitted function
plt.figure()
plt.plot(x_in5, x_min[0] * np.cos(x_min[1]*x_in5) + x_min[1] * np.sin(x_min[0]*x_in5), 'b-')
plt.plot(x_in5, y_out5, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

plt.figure()
X = np.linspace(x_min[0] - 1, x_min[0] + 1, 500)
Y = np.array([objective_fn5(np.array([x, x_min[1]])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[0], info["obj_value"], "rx")
plt.title("Plot along x[0]")
plt.show()

plt.figure()
X = np.linspace(x_min[1] - 1, x_min[1] + 1, 500)
Y = np.array([objective_fn5(np.array([x_min[0], x])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[1], info["obj_value"], "rx")
plt.title("Plot along x[1]")
plt.show()

In [ ]:
# Function fitting #5 with gauss newton

x0 = np.array([100.01, 102.01])
max_iterations = 1000
x_min, info = min_gauss_newton(residuals_fn5, residuals_jacobian_fn5, x0, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(objective_fn5(x_min)))

# Plot data and fitted function
plt.figure()
plt.plot(x_in5, x_min[0] * np.cos(x_min[1]*x_in5) + x_min[1] * np.sin(x_min[0]*x_in5), 'b-')
plt.plot(x_in5, y_out5, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

plt.figure()
X = np.linspace(x_min[0] - 1, x_min[0] + 1, 500)
Y = np.array([objective_fn5(np.array([x, x_min[1]])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[0], info["obj_value"], "rx")
plt.title("Plot along x[0]")
plt.show()

plt.figure()
X = np.linspace(x_min[1] - 1, x_min[1] + 1, 500)
Y = np.array([objective_fn5(np.array([x_min[0], x])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[1], info["obj_value"], "rx")
plt.title("Plot along x[1]")
plt.show()

In [ ]:
# Function fitting #5 with levenberg-marquardt

x0 = np.array([100.01, 102.03])
max_iterations = 50
x_min, info = min_levenberg_marquardt(residuals_fn5, residuals_jacobian_fn5, x0, max_iterations=max_iterations)
print("minimum x: {}".format(x_min))
print("objective value: {}".format(objective_fn5(x_min)))

# Plot data and fitted function
plt.figure()
plt.plot(x_in5, x_min[0] * np.cos(x_min[1]*x_in5) + x_min[1] * np.sin(x_min[0]*x_in5), 'b-')
plt.plot(x_in5, y_out5, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

plt.figure()
X = np.linspace(x_min[0] - 1, x_min[0] + 1, 500)
Y = np.array([objective_fn5(np.array([x, x_min[1]])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[0], info["obj_value"], "rx")
plt.title("Plot along x[0]")
plt.show()

plt.figure()
X = np.linspace(x_min[1] - 1, x_min[1] + 1, 500)
Y = np.array([objective_fn5(np.array([x_min[0], x])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[1], info["obj_value"], "rx")
plt.title("Plot along x[1]")
plt.show()

In [ ]:
# Minimize function #1 with scipy BFGS
x0 = np.array([100.01, 102.03])
import scipy.optimize
res = scipy.optimize.minimize(objective_fn5, x0, method="BFGS", jac=grad_fn5)
x_min = res.x
print("minimum x: {}".format(x_min))
print("objective value: {}".format(res.fun))

# Plot data and fitted function
plt.figure()
plt.plot(x_in5, x_min[0] * np.cos(x_min[1]*x_in5) + x_min[1] * np.sin(x_min[0]*x_in5), 'b-')
plt.plot(x_in5, y_out5, 'r-')
plt.legend(["Fit", "Data"])
plt.show()

plt.figure()
X = np.linspace(x_min[0] - 1, x_min[0] + 1, 500)
Y = np.array([objective_fn5(np.array([x, x_min[1]])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[0], info["obj_value"], "rx")
plt.title("Plot along x[0]")
plt.show()

plt.figure()
X = np.linspace(x_min[1] - 1, x_min[1] + 1, 500)
Y = np.array([objective_fn5(np.array([x_min[0], x])) for x in X])
plt.plot(X, Y)
plt.plot(x_min[1], info["obj_value"], "rx")
plt.title("Plot along x[1]")
plt.show()